In [2]:
import PyPDF2 as pdf
from tabula import read_pdf
from tabulate import tabulate
import pandas as pd
import os
import re

In [3]:
def gener_gene_data(pdf_path):
    #read in pdf and get the table
    pdf = read_pdf(pdf_path, pages=1)
    pd_df = pdf[1]
    pd_df = pd_df.rename(columns=pd_df.iloc[0]).drop(pd_df.index[0])

    #get the transcript IDs from pdf
    id_list = list(pd_df["TRANSCRIPT ID"])
    id_list = [x.split('.')[0] for x in id_list]
    patient_name = '-'.join(pdf_path.split('_')[:2])
    
    print("Patient Name: " + patient_name)
    print("Transcript Ids being retrieved: " + str(id_list))
    
    #find corresponding excel file based on patient name
    xl_files_og = [f for f in os.listdir('.') if os.path.isfile(f) and f.endswith('.xlsx')]
    xl_files = [f.replace('%', '_') for f in xl_files_og]
    xl_files = [f.split('_')[0] for f in xl_files]
    ind = 0
    for i in range(0, len(xl_files)):
        if patient_name == xl_files[i]:
            ind = i
    
    # read excel file and filter based on transcript IDs list
    print("Reading from " + xl_files_og[ind])
    gene_data = pd.read_excel(xl_files_og[ind])
    # gene_data["Variant Filter"] = gene_data.apply(lambda s: s.Variant.split(' ')[0], axis=1)
    gd = gene_data.loc[gene_data["TranscriptID"].isin(id_list)]
    # print("table made")
    return patient_name, gd  

# gener_gene_data("CCGL_3205_FinalReport_pedi_LFS_associated_diffuse_astrocytoma_IDH1_R132H_germline_TP53.pdf")

In [8]:
files = [f for f in os.listdir('.') if os.path.isfile(f) and f.endswith('.pdf')]
# print("using the following pdf files: \n" + str(files))
names = []
reports = []
for f in files:
    x = gener_gene_data(f)
    names.append(x[0])
    reports.append(x[1])

combo = zip(names, reports)
data_reports = {n:r for n,r in combo}

#writing to directory
if not os.path.exists("Results"):
    os.mkdir("Results")
for n in names:
    data_reports.get(n).to_csv("Results/" + n)
os.listdir("Results")

Patient Name: CCGL-3152
Transcript Ids being retrieved: ['NM_000489', 'all', 'NM_005896']
Reading from CCGL-3152_CGP-9473_T_signout.xlsx
Patient Name: CCGL-3149
Transcript Ids being retrieved: ['NM_000489', 'all', 'NM_005896', 'all', 'NM_000546']
Reading from CCGL-3149_CGP-9470_T_signout.xlsx
Patient Name: CCGL-2831
Transcript Ids being retrieved: ['NM_000489', 'NM_005896', 'NM_000546']
Reading from CCGL-2831_CGP-8784_T_signout.xlsx
Patient Name: CCGL-2835
Transcript Ids being retrieved: ['NM_000489', 'NM_005896', 'NM_181523', 'NM_000546']
Reading from CCGL-2835_CGP-8785_T_signout.xlsx
Patient Name: CCGL-3205
Transcript Ids being retrieved: ['NM_005896', 'NM_000546']
Reading from CCGL-3205%2CCCGL-1628_CGP-9586_T_CGP-6259_N_signout.xlsx
Patient Name: CCGL-3619
Transcript Ids being retrieved: ['NM_000489', 'NM_000489', 'NM_000489', 'NM_005896', 'NM_001042492', 'NM_017617', 'NM_152594', 'NM_000546']
Reading from CCGL-3619_CGP-10495_T_CGP-10511_N_signout.xlsx


['CCGL-3205', 'CCGL-2835', 'CCGL-3619', 'CCGL-3149', 'CCGL-2831', 'CCGL-3152']

In [49]:
# aggregating all results by reading in each csv file, appending to DF and adding CCGL
files = os.listdir("Results")
print(files)
patient_csvs = []
for file in files:
    csv = pd.read_csv("Results/" + file)
    csv.insert(loc=0, column='CCGL', value=file)
    patient_csvs.append(csv)

aggr = pd.concat(patient_csvs)

#change based on which columns to keep
cols_to_keep = ['CCGL', 'CHROM', 'POS', 'REF', 'ALT', 'Func', 
       'Variant', 'TranscriptID', 'Classification', 'T:COV', 'T:MAF', 'T:LogR',
       'GeneDetail', 'ExonicFunc',
       'AAChange', 'ExAC', 'gnomADexome', 'esp6500si', '1000g2015aug',
       'snp150', 'ucsf500normT', 'ucsf500normN', 'cosmic87',
       'clinvar', 'CADD']
aggr = aggr.filter(cols_to_keep)

#write to directory
aggr.to_csv("Aggregate Data")


['CCGL-3205', 'CCGL-2835', 'CCGL-3619', 'CCGL-3149', 'CCGL-2831', 'CCGL-3152']
